In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import sys
sys.path.append('/Users/wastechs/Documents/git-repos/energy-efficiency')
from lib.util.helper import query_table, weekday_time_series
import seaborn as sns
import plotly.express as px
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot

In [2]:
druck = weekday_time_series(sensor_id='xl106_druckmaschine_5T')

In [3]:
druck['kw'] = round(druck['kw'], 2)

In [4]:
px.line(
    x=druck.index, y=druck.kw, 
    title='UV Sigma Line',
    labels={
        'x': 'Time',
        'y': 'kW'
    },
    markers=False)

Positive skew in the variance of the time series could be a problem as GAMs assume Gaussian noise of the residuals
 - How to handle when the machine is off? (0 values)

In [188]:
str(druck.index.date.max())

'2021-10-15'

In [30]:
day_range = druck.index.max() - druck.index.min()
day_range

Timedelta('4 days 23:55:00')

In [31]:
druck_final = pd.DataFrame(data=druck['kw'], index=druck.index)
#druck_final['hour'] = druck_final.index.hour
druck_final.reset_index(inplace=True)

# Scaling helps with convergence via MCMC
scale = StandardScaler()
druck_final['kw'] = scale.fit_transform(druck_final['kw'].values.reshape(-1, 1))

#druck_test = druck_line[druck_line.index.day >= 17]
#druck_test_final = pd.DataFrame(data=druck_test['kw'], index=druck_test.index)